In [49]:
# importing the libraries
import pandas as pd
import numpy as np

# for reading and displaying images
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline

# for creating validation set
from sklearn.model_selection import train_test_split

# for evaluating the model
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD

# importing paths
import os.path
from os import path

# import image editing tools
from PIL import Image
import cv2
from skimage.measure import block_reduce

#from tqdm.notebook import tqdm


In [50]:
# loading dataset
train = pd.read_csv('./data/data.csv')
#test = pd.read_csv('test_ScVgIM0/test.csv')

#sample_submission = pd.read_csv('sample_submission_I5njJSF.csv')

train.head()

,image,object_xyz
0,0.jpg,[ 0.945272 0.02409354 0.02173636 -0.063459...
1,1.jpg,[ 0.94526975 0.02408728 0.02175264 -0.063460...
2,2.jpg,[ 0.94527507 0.02409035 0.02174615 -0.063465...
3,3.jpg,[ 0.94537809 0.02417826 0.02184221 -0.063134...
4,4.jpg,[ 0.9453916 0.02418706 0.02183683 -0.063146...


In [51]:
# loading training images
train_img = []
train_xyz = []

test = 0

for img_name in tqdm(train['image']):

    # TODO: change the path
    image_path = "/Users/riadoshi/Downloads/convnet/data/" + str(img_name)
        
    if(path.exists(image_path)):
        
        ## IMAGE EDITING
        
        img = imread(image_path, as_gray=True) # read the image
        img = img[0:240, 40:280] # crop the image
        img = block_reduce(img, block_size=(3,3), func=np.mean)
    
        img /= 255.0 # normalize the pixel values
        img = img.astype('float32') # converting the type of pixel to float 32
        train_img.append(img)
        test+=1
        #cv2.imshow('hi',img)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()


print(test)
train_xyz = []
counter = 0
for xyz in train['object_xyz'].values:
    img_name = train['image'][counter]
    image_path = "/Users/riadoshi/Downloads/convnet/data/" + str(img_name)
    if(path.exists(image_path)):
        f = lambda x: x.strip('][').split(' ')
        xyz_fixed = [float(i) for i in f(xyz) if i][:3]
        train_xyz.append(xyz_fixed)
    counter+=1

# converting the list to numpy array
train_x1 = np.array(train_img)
train_y1 = np.array(train_xyz)

# defining the target
#train_y = train['object_xyz'].values
#train_x.shape


100%|██████████| 44454/44454 [06:16<00:00, 118.22it/s]  


39778


In [52]:
train_x1.shape

(39778, 80, 80)

In [66]:
# create validation set
train_x, val_x, train_y, val_y = train_test_split(train_x1, train_y1, test_size = 0.2)
(train_x.shape, train_y.shape), (val_x.shape, val_y.shape)

(((31822, 80, 80), (31822, 3)), ((7956, 80, 80), (7956, 3)))

In [68]:
# converting training images into torch format
train_x = train_x.reshape(31822, 1, 80, 80)
train_x  = torch.from_numpy(train_x)

# converting the target into torch format
train_y = train_y.astype(int);
train_y = torch.from_numpy(train_y)

# shape of training data
train_x.shape, train_y.shape

(torch.Size([31822, 1, 80, 80]), torch.Size([31822, 3]))

In [69]:
# converting validation images into torch format
val_x = val_x.reshape(7956, 1, 80, 80)
val_x  = torch.from_numpy(val_x)

# converting the target into torch format
val_y = val_y.astype(int);
val_y = torch.from_numpy(val_y)

# shape of validation data
val_x.shape, val_y.shape

(torch.Size([7956, 1, 80, 80]), torch.Size([7956, 3]))

In [70]:
# conv net
class Net(Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            Conv2d(1, 4, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(4),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            # Defining another 2D convolution layer
            Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(4),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
        )

        self.linear_layers = Sequential(
            Linear(4 * 79 * 79, 3)
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [63]:
# defining the model
model = Net()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.07)
# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    
print(model)

Net(
  (cnn_layers): Sequential(
    (0): Conv2d(3, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=196, out_features=10, bias=True)
  )
)


In [64]:
def train(epoch):
    model.train()
    tr_loss = 0
    # getting the training set
    x_train, y_train = Variable(train_x), Variable(train_y)
    # getting the validation set
    x_val, y_val = Variable(val_x), Variable(val_y)
    # converting the data into GPU format
    if torch.cuda.is_available():
        x_train = x_train.cuda()
        y_train = y_train.cuda()
        x_val = x_val.cuda()
        y_val = y_val.cuda()

    # clearing the Gradients of the model parameters
    optimizer.zero_grad()
    
    # prediction for training and validation set
    output_train = model(x_train)
    output_val = model(x_val)

    # computing the training and validation loss
    loss_train = criterion(output_train, y_train)
    loss_val = criterion(output_val, y_val)
    train_losses.append(loss_train)
    val_losses.append(loss_val)

    # computing the updated weights of all the model parameters
    loss_train.backward()
    optimizer.step()
    tr_loss = loss_train.item()
    if epoch%2 == 0:
        # printing the validation loss
        print('Epoch : ',epoch+1, '\t', 'loss :', loss_val)

In [65]:
# defining the number of epochs
n_epochs = 25
# empty list to store training losses
train_losses = []
# empty list to store validation losses
val_losses = []
# training the model
#for epoch in range(n_epochs):
    #train(epoch)

RuntimeError: Given groups=1, weight of size [4, 3, 3, 3], expected input[31822, 1, 80, 80] to have 3 channels, but got 1 channels instead